In [62]:
#####################################################################
# Examples below are (more or less) taken from 
# “Python for Data Analysis" by William Wesley McKinney (O’Reilly).
# Copyright 2012 William McKinney, 978-1-449-31979-3.
#
# TOPICS COVERED:
# - Combining & Merging Datasets
# - Reshaping & Pivoting
# - Data Transformation
#####################################################################

# This notebook is written in Python 3.

from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from numpy import nan as NA

## Combining & Merging Datasets

### Merging

In [63]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [64]:
# If not specified, pd.merge uses the overlapping column names as the keys
# Good practice is to specify the columns explicitly
pd.merge(df1, df2)           # OK but not a good idea
pd.merge(df1, df2, on='key') # Same result as above but better code

# Take note of how data2 column is being filled up.

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [65]:
# In the event that the column names are not the same, you can specify the column
# names which should be merged on explicitly with left_on and right_on
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [66]:
# Note in the above that c & d for key have dropped out.
# By default, merge does an 'inner' join (i.e. only intersection is taken).
# Other options are 'left', 'right' and 'outer' joins.
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,NaN
7,NaN,d,2


In [67]:
# Many-to-many merges have well-defined but not necessarily intuitive behaviour!
# It ends up giving a cartesian product of the rows.
df5 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df6 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
print(df5)
print(df6)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b
   data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d


In [68]:
pd.merge(df5, df6, on='key', how='left')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


In [69]:
# To merge with multiple keys, pass a list of column names
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
print(left)
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [70]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


In [71]:
# Notice that because both DataFrames have 'key2' column, merging them produces
# 2 columns named "key2_x" and "key2_y".
print(pd.merge(left, right, on='key1'))

# You can change the naming convention with the suffixes option
print(pd.merge(left, right, on='key1', suffixes=('_left', '_right')))

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


In [72]:
# To merge on index, can pass left_index = True or right_index = True (or both).
# Alternatively, can use join method:
# E.g. left.join(right, how='outer')

### Concatenating on Axis

In [73]:
arr = np.arange(12).reshape((3, 4))
print(arr)
print(np.concatenate([arr, arr], axis = 0))  # stack on top of each other
print(np.concatenate([arr, arr], axis = 1))  # stack side by side

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]


In [74]:
s1 = Series([0, 1], index=['a', 'b'])
s4 = Series([0, 5, 5, 6], index=['a', 'b', 'c', 'd'])
print(pd.concat([s1, s4], axis=1))  # 'outer' join, NaNs will appear
print(pd.concat([s1, s4], axis=1, join='inner'))  # 'inner' join, returns intersect only

    0  1
a   0  0
b   1  5
c NaN  5
d NaN  6
   0  1
a  0  0
b  1  5


### Combine

In [75]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df1

,a,b,c
0,1,NaN,2
1,NaN,2,6
2,5,NaN,10
3,NaN,6,14


In [76]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5,NaN
1,4,3
2,NaN,4
3,3,6
4,7,8


In [77]:
df1.combine_first(df2)  # where there are NaNs in df1, patch with the value in df2

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,6,14
4,7,8,NaN


## Reshaping & Pivoting

In [78]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
result = data.stack()  # pivots the cols into rows, making a Series
result

# Note that stacking removes missing data by default

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [80]:
# unstack gives back the DataFrame
print(result.unstack())   # by default, innermost level is unstacked.
print(result.unstack(0))  # can unstack by other levels too
print(result.unstack('state'))  # same result as line above

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/index.py:4281: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [81]:
ldata = pd.read_csv("Datasets/ldata.csv")
ldata

,date,item,value
0,1959-03-31 00:00:00,realgdp,2710.349
1,1959-03-31 00:00:00,infl,0.000
2,1959-03-31 00:00:00,unemp,5.800
3,1959-06-30 00:00:00,realgdp,2778.801
4,1959-06-30 00:00:00,infl,2.340
5,1959-06-30 00:00:00,unemp,5.100
6,1959-09-30 00:00:00,realgdp,2775.488
7,1959-09-30 00:00:00,infl,2.740
8,1959-09-30 00:00:00,unemp,5.300
9,1959-12-31 00:00:00,realgdp,2785.204


In [82]:
# The first 2 values are the columns to be used as row & column index
# last is an optional value column to fill the DataFrame
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 00:00:00,0.00,2710.349,5.8
1959-06-30 00:00:00,2.34,2778.801,5.1
1959-09-30 00:00:00,2.74,2775.488,5.3
1959-12-31 00:00:00,NaN,2785.204,NaN


In [83]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,value,value2
0,1959-03-31 00:00:00,realgdp,2710.349,0.958651
1,1959-03-31 00:00:00,infl,0.000,-0.258143
2,1959-03-31 00:00:00,unemp,5.800,0.277850
3,1959-06-30 00:00:00,realgdp,2778.801,-0.110210
4,1959-06-30 00:00:00,infl,2.340,2.529435
5,1959-06-30 00:00:00,unemp,5.100,1.501763
6,1959-09-30 00:00:00,realgdp,2775.488,0.284364
7,1959-09-30 00:00:00,infl,2.740,-1.140385
8,1959-09-30 00:00:00,unemp,5.300,-1.217307
9,1959-12-31 00:00:00,realgdp,2785.204,1.406905


In [84]:
# With more than one value column, pivoting results in hierarchical columns
print(ldata.pivot('date', 'item'))

                    value                    value2                    
item                 infl   realgdp unemp      infl   realgdp     unemp
date                                                                   
1959-03-31 00:00:00  0.00  2710.349   5.8 -0.258143  0.958651  0.277850
1959-06-30 00:00:00  2.34  2778.801   5.1  2.529435 -0.110210  1.501763
1959-09-30 00:00:00  2.74  2775.488   5.3 -1.140385  0.284364 -1.217307
1959-12-31 00:00:00   NaN  2785.204   NaN       NaN  1.406905       NaN


## Data Transformation

In [85]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
print(data)
print(data.duplicated())  # boolean array indicating whether row is a duplicate
print(data.drop_duplicates())  # drops the duplicates

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4
0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool
    k1  k2
0  one   1
2  one   2
3  two   3
5  two   4


In [86]:
# by default, the methods above consider all columns.
print(data.drop_duplicates(['k1']))  # drop duplicates when considering only column 'k1'
print(data.drop_duplicates(['k1'],
                           keep='last'))  # instead of taking the first row, take the last

    k1  k2
0  one   1
3  two   3
    k1  k2
2  one   2
6  two   4


In [87]:
# use map function to do element-wise transformation
data['k3'] = data['k2'].map(lambda x: x**2)
data

,k1,k2,k3
0,one,1,1
1,one,1,1
2,one,2,4
3,two,3,9
4,two,3,9
5,two,4,16
6,two,4,16


In [88]:
# use replace method to replace data
print(data.replace(1, NA))
print(data.replace({1:NA, 2:"hi"}))

    k1  k2  k3
0  one NaN NaN
1  one NaN NaN
2  one   2   4
3  two   3   9
4  two   3   9
5  two   4  16
6  two   4  16
    k1   k2  k3
0  one  NaN NaN
1  one  NaN NaN
2  one   hi   4
3  two    3   9
4  two    3   9
5  two    4  16
6  two    4  16


In [89]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
# If I want bins of 18 to 25, 26 to 35, 35 to 60, and 60 and older:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
print(cats)

# can also label the groups
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]


[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [90]:
pd.cut(ages, bins, right=False)  # notice the different ends for the intervals

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, object): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [91]:
print(pd.cut(ages, 4))   # cut in 4 equal parts based on max & min
print(pd.qcut(ages, 4))  # cut in 4 parts based on percentiles

[(19.959, 30.25], (19.959, 30.25], (19.959, 30.25], (19.959, 30.25], (19.959, 30.25], ..., (30.25, 40.5], (50.75, 61], (40.5, 50.75], (40.5, 50.75], (30.25, 40.5]]
Length: 12
Categories (4, object): [(19.959, 30.25] < (30.25, 40.5] < (40.5, 50.75] < (50.75, 61]]
[[20, 22.75], [20, 22.75], (22.75, 29], (22.75, 29], [20, 22.75], ..., (29, 38], (38, 61], (38, 61], (38, 61], (29, 38]]
Length: 12
Categories (4, object): [[20, 22.75] < (22.75, 29] < (29, 38] < (38, 61]]


In [92]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [93]:
# to produce a random permutation of the rows
sampler = np.random.permutation(len(df))
print(sampler)
print(df.take(sampler))

[0 4 1 2 3]
    0   1   2   3
0   0   1   2   3
4  16  17  18  19
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15


In [94]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [95]:
# get dummy variables for the column 'key'
pd.get_dummies(df['key'], prefix='key')

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0
